In [ ]:
import vggish
import vggish.vggish_params as vggish_params
import vggish.vggish_slim as vggish_slim
import tensorflow as tf

# Te downloaden van https://storage.googleapis.com/audioset/vggish_model.ckpt
CHECKPOINT_FILE = './vggish_model.ckpt'

# Definieer tensorflow sessie
with tf.Graph().as_default(), tf.Session() as sess:
    # Definieer het VVGish model
    vggish_slim.define_vggish_slim(training=False)
    # Laad de checpoint
    vggish_slim.load_vggish_slim_checkpoint(sess, CHECKPOINT_FILE)
    
    # Identificeer input/output tensors
    features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.OUTPUT_TENSOR_NAME)
    
    # Initialiseer tflite converter
    converter = tf.lite.TFLiteConverter.from_session(sess, [features_tensor], [embedding_tensor])
    
    # Converteer model as-is naar tflite formaat
    converter.post_training_quantize=False
    open("converted_model.tflite", "wb").write(converter.convert())
    
    # Quantizeer model (verkleint groote met factor 4, verlaagt mogelijk de kwaliteit)
    converter.post_training_quantize=True
    open("converted_model_quantized.tflite", "wb").write(converter.convert())